In [1]:
import os
from data_utils import *
from model_utils import *
from interval_model import *
from train_test import *
import gc
import numpy as np

In [2]:
word_subs = 'word_subs1.json'

train_dataloader, test_dataloader = load_dataloader('', imdb=True, word_subs=word_subs, batch_size=64)

data = next(iter(train_dataloader))
data[0].shape, data[1].shape

(torch.Size([64, 256]), torch.Size([64]))

# Verified Training

In [3]:
base_model = BOWModel(train_dataloader.dataset.embeddings, train_dataloader.dataset.pad_idx)
base_model.to('cuda')
# state_dict = torch.load('base_model1.pth')
# base_model.load_state_dict(state_dict)
interval_model = IntervalModel(base_model, mur_computation=True).to('cuda')

In [4]:
epoch_times = train_text_verified(base_model, interval_model, train_dataloader, train_dataloader.dataset, num_epochs=10, elide_last=True)

print(f"Total Time: {np.sum(epoch_times)}, Average Time: {np.mean(epoch_times)}, Std Time: {np.std(epoch_times)}")

Time for loop: 4.5178382396698
Epoch 0: Loss: 0.4663, Accuracy: 77.18%, Verified Accuracy: 0.00%, epsilon: 0.0000, kappa: 1.00, Time: 4.52s
Time for loop: 417.6245810985565
Epoch 1: Loss: 0.3935, Accuracy: 82.56%, Verified Accuracy: 78.12%, epsilon: 0.0111, kappa: 0.94, Time: 417.62s
Time for loop: 381.815758228302
Epoch 2: Loss: 0.3783, Accuracy: 84.32%, Verified Accuracy: 77.10%, epsilon: 0.0222, kappa: 0.89, Time: 381.82s
Time for loop: 411.56813502311707
Epoch 3: Loss: 0.3749, Accuracy: 84.98%, Verified Accuracy: 76.49%, epsilon: 0.0333, kappa: 0.83, Time: 411.57s
Time for loop: 453.78313302993774
Epoch 4: Loss: 0.3832, Accuracy: 85.34%, Verified Accuracy: 75.29%, epsilon: 0.0444, kappa: 0.78, Time: 453.78s
Time for loop: 366.7028806209564
Epoch 5: Loss: 0.3936, Accuracy: 85.77%, Verified Accuracy: 73.77%, epsilon: 0.0556, kappa: 0.72, Time: 366.70s
Time for loop: 363.1786513328552
Epoch 6: Loss: 0.4086, Accuracy: 85.92%, Verified Accuracy: 73.01%, epsilon: 0.0667, kappa: 0.67, Tim

In [5]:
for eps in np.arange(0.01, 0.11, 0.01):
    print(f"Testing on eps: {eps}")
    train_text_verified(base_model, interval_model, test_dataloader, train_dataloader.dataset, num_epochs=1, elide_last=False, eps=eps, test_mode=True)

Testing on eps: 0.01
Time for loop: 470.7357888221741
Epoch 0: Loss: 0.3704, Accuracy: 83.88%, Verified Accuracy: 82.71%, epsilon: 0.0100, kappa: 1.00, Time: 470.74s
Testing on eps: 0.02
Time for loop: 375.9930899143219
Epoch 0: Loss: 0.3704, Accuracy: 83.88%, Verified Accuracy: 81.46%, epsilon: 0.0200, kappa: 1.00, Time: 375.99s
Testing on eps: 0.03
Time for loop: 371.2543978691101
Epoch 0: Loss: 0.3704, Accuracy: 83.88%, Verified Accuracy: 80.27%, epsilon: 0.0300, kappa: 1.00, Time: 371.25s
Testing on eps: 0.04
Time for loop: 381.90950560569763
Epoch 0: Loss: 0.3704, Accuracy: 83.88%, Verified Accuracy: 78.98%, epsilon: 0.0400, kappa: 1.00, Time: 381.91s
Testing on eps: 0.05
Time for loop: 371.26103258132935
Epoch 0: Loss: 0.3704, Accuracy: 83.88%, Verified Accuracy: 77.70%, epsilon: 0.0500, kappa: 1.00, Time: 371.26s
Testing on eps: 0.060000000000000005
Time for loop: 383.1868233680725
Epoch 0: Loss: 0.3704, Accuracy: 83.88%, Verified Accuracy: 76.26%, epsilon: 0.0600, kappa: 1.00, 

# Standard Training

In [3]:
base_model = BOWModel(train_dataloader.dataset.embeddings, train_dataloader.dataset.pad_idx)
base_model.to('cuda')
state_dict = torch.load('base_modeltext.pth')
base_model.load_state_dict(state_dict)
interval_model = IntervalModel(base_model, mur_computation=True).to('cuda')

In [4]:
epoch_times = train_text_verified(base_model, interval_model, train_dataloader, train_dataloader.dataset, num_epochs=10, elide_last=False, eps=0.0)

print(f"Total Time: {np.sum(epoch_times)}, Average Time: {np.mean(epoch_times)}, Std Time: {np.std(epoch_times)}")

Time for loop: 5.028216123580933
Epoch 0: Loss: 0.4729, Accuracy: 76.28%, Verified Accuracy: 0.00%, epsilon: 0.0000, kappa: 1.00, Time: 5.03s
Time for loop: 4.6910786628723145
Epoch 1: Loss: 0.3611, Accuracy: 83.08%, Verified Accuracy: 0.00%, epsilon: 0.0000, kappa: 0.94, Time: 4.69s
Time for loop: 4.774171590805054
Epoch 2: Loss: 0.3196, Accuracy: 84.47%, Verified Accuracy: 0.00%, epsilon: 0.0000, kappa: 0.89, Time: 4.77s
Time for loop: 5.586472034454346
Epoch 3: Loss: 0.2869, Accuracy: 85.24%, Verified Accuracy: 0.00%, epsilon: 0.0000, kappa: 0.83, Time: 5.59s
Time for loop: 5.413620948791504
Epoch 4: Loss: 0.2503, Accuracy: 86.45%, Verified Accuracy: 0.00%, epsilon: 0.0000, kappa: 0.78, Time: 5.41s
Time for loop: 5.325513601303101
Epoch 5: Loss: 0.2217, Accuracy: 87.13%, Verified Accuracy: 0.00%, epsilon: 0.0000, kappa: 0.72, Time: 5.33s
Time for loop: 5.329435348510742
Epoch 6: Loss: 0.1906, Accuracy: 88.23%, Verified Accuracy: 0.00%, epsilon: 0.0000, kappa: 0.67, Time: 5.33s
Time 

In [4]:
for eps in np.arange(0.01, 0.11, 0.01):
    print(f"Testing on eps: {eps}")
    train_text_verified(base_model, interval_model, test_dataloader, train_dataloader.dataset, num_epochs=1, elide_last=False, eps=eps, test_mode=True)

Testing on eps: 0.01
Time for loop: 480.2958381175995
Epoch 0: Loss: 0.3617, Accuracy: 84.66%, Verified Accuracy: 77.36%, epsilon: 0.0100, kappa: 1.00, Time: 480.30s
Testing on eps: 0.02
Time for loop: 452.1203875541687
Epoch 0: Loss: 0.3617, Accuracy: 84.66%, Verified Accuracy: 68.31%, epsilon: 0.0200, kappa: 1.00, Time: 452.12s
Testing on eps: 0.03
Time for loop: 430.96066427230835
Epoch 0: Loss: 0.3617, Accuracy: 84.66%, Verified Accuracy: 58.10%, epsilon: 0.0300, kappa: 1.00, Time: 430.96s
Testing on eps: 0.04
Time for loop: 498.5187211036682
Epoch 0: Loss: 0.3617, Accuracy: 84.66%, Verified Accuracy: 47.62%, epsilon: 0.0400, kappa: 1.00, Time: 498.52s
Testing on eps: 0.05
Time for loop: 447.2361719608307
Epoch 0: Loss: 0.3617, Accuracy: 84.66%, Verified Accuracy: 37.61%, epsilon: 0.0500, kappa: 1.00, Time: 447.24s
Testing on eps: 0.060000000000000005
Time for loop: 594.6508650779724
Epoch 0: Loss: 0.3617, Accuracy: 84.66%, Verified Accuracy: 28.94%, epsilon: 0.0600, kappa: 1.00, T